In [5]:
import numpy as np
import pandas as pd

# Function to check if a point is inside the blue region
def is_in_blue_region(x1, x2):
    # Equations of the lines
    line_1 = (10 * x1 + 4 * x2 <= 80)
    line_2 = (10 * x1 - 4 * x2 <= 20)
    line_3 = (x2 <= 6)
    line_4 = (7 * x1 + 10 * x2 >= 80)
    line_5 = (-7 * x1 + 10 * x2 >= 10)
    
    result = line_1 and line_2 and line_3 and line_4 and line_5
    # Check if the point lies within all boundaries (inside the blue region)
    return result

# Set the number of points for each label (0 and 1)
n_points_per_label = 5000

# Create lists to store data points
data_0 = []
data_1 = []

# Generate random points and classify them
while len(data_0) < n_points_per_label or len(data_1) < n_points_per_label:
    # Randomly sample x1 and x2 values in the range [0, 10) as floating-point values
    x1 = np.random.uniform(0, 10)
    x2 = np.random.uniform(0, 10)
    
    # Determine whether the point is inside the blue region
    if is_in_blue_region(x1, x2):
        label = 1  # Inside the blue region
        if len(data_1) < n_points_per_label:
            data_1.append([x1, x2, label])  # Append only if we need more 1s
    else:
        label = 0  # Outside the blue region
        if len(data_0) < n_points_per_label:
            data_0.append([x1, x2, label])  # Append only if we need more 0s

# Combine both label groups into a single dataset
data = data_0 + data_1
np.random.shuffle(data)  # Shuffle to ensure random distribution

# Convert the data to a DataFrame
df = pd.DataFrame(data, columns=['x1', 'x2', 'label'])

# Save to a CSV file
df.to_csv('./blue_region_data2.csv', index=False)

print(f"Dataset created with {n_points_per_label} points for each label (0 and 1).")


Dataset created with 5000 points for each label (0 and 1).


In [9]:
import numpy as np
import pandas as pd

# Define the threshold function as specified
def threshold(x, t):
    return 1 if x >= t else 0

# Custom Neural Network class
class CustomNN:
    def __init__(self):
        # Manually initialize weights and threshold values
        self.weights = [
            np.array([[-10, -4], [-10, 4], [0, -1], [7, 10], [-7, 10]]),  # Weights for the first hidden layer
            np.array([[1, 1, 1, 1, 1]])  # Weights for the output layer
        ]
        self.thresholds = [
            np.array([-80, -20, -6, 80, 10]),  # Thresholds for the first hidden layer
            np.array([5])  # Threshold for the output layer
        ]

    def forward(self, X):
        # First hidden layer computation
        hidden1 = np.zeros((X.shape[0], 5))
        
        # Compute activations for the first hidden layer
        for i in range(5):  # Adjusted to loop through all 5 neurons
            weighted_sum = X @ self.weights[0][i]
            hidden1[:, i] = [threshold(x, self.thresholds[0][i]) for x in weighted_sum]
        
        # Output layer computation
        output = np.zeros((X.shape[0], 1))
        weighted_sum = hidden1 @ self.weights[1].T  # Use the correct weights for the output layer
        output[:, 0] = [threshold(x, self.thresholds[1][0]) for x in weighted_sum]

        return output

# Example usage
if __name__ == "__main__":
    # Load the generated dataset
    df = pd.read_csv('blue_region_data2.csv')

    # Extract the input (x1, x2) and target output columns
    X = df[['x1', 'x2']].values
    y_true = df['label'].values  # Assuming 'output' column is 1 for blue region, 0 for non-blue region

    # Instantiate and run the custom neural network
    model = CustomNN()
    outputs = model.forward(X)

    # Compare the predicted output with the true output
    correct_predictions = np.sum(outputs.flatten() == y_true)
    accuracy = correct_predictions / len(y_true) * 100

    print(f"Accuracy of the neural network on the dataset: {accuracy:.2f}%")

    # Print some sample predictions
    print("\nSample predictions:")
    for i in range(5):  # Show the first 5 predictions
        print(f"Input: {X[i]}, Predicted Output: {outputs[i][0]}, True Output: {y_true[i]}")


Accuracy of the neural network on the dataset: 100.00%

Sample predictions:
Input: [7.81554838 7.70706619], Predicted Output: 0.0, True Output: 0
Input: [3.77916618 5.37231266], Predicted Output: 1.0, True Output: 1
Input: [1.99922609 4.53671549], Predicted Output: 0.0, True Output: 0
Input: [3.04631057 5.96338812], Predicted Output: 1.0, True Output: 1
Input: [2.56790674 2.42705017], Predicted Output: 0.0, True Output: 0


In [8]:
import numpy as np
import pandas as pd

# Define the threshold function as specified
def threshold(x, t):
    return 1 if x >= t else 0

# Custom Neural Network class
class CustomNN:
    def __init__(self):
        # Manually initialize weights and threshold values
        self.weights = [
            np.array([[-1, -1], [1, 1], [1, 1], [-1, -1]]),  # Weights for the first hidden layer
            np.array([[1, 1, 0, 0], [0, 0, 1, 1]]),          # Weights for the second hidden layer
            np.array([[1, 1]])                                # Weights for the output layer
        ]
        self.thresholds = [
            np.array([-1, 9, 4, -6]),   # Thresholds for the first hidden layer
            np.array([1, 2]),           # Thresholds for the second hidden layer
            np.array([1])               # Threshold for the output layer
        ]

    def forward(self, X):
        # First hidden layer computation
        hidden1 = np.zeros((X.shape[0], 4))
        for i in range(4):
            weighted_sum = X @ self.weights[0][i]
            hidden1[:, i] = [threshold(x, self.thresholds[0][i]) for x in weighted_sum]
        
        # Second hidden layer computation
        hidden2 = np.zeros((X.shape[0], 2))
        for i in range(2):
            weighted_sum = hidden1 @ self.weights[1][i]
            hidden2[:, i] = [threshold(x, self.thresholds[1][i]) for x in weighted_sum]

        # Output layer computation
        output = np.zeros((X.shape[0], 1))
        weighted_sum = hidden2 @ self.weights[2][0]
        output[:, 0] = [threshold(x, self.thresholds[2][0]) for x in weighted_sum]

        return output

# Example usage
if __name__ == "__main__":
    # Load the generated dataset
    df = pd.read_csv('blue_region_data1.csv')

    # Extract the input (x1, x2) and target output columns
    X = df[['x1', 'x2']].values
    y_true = df['label'].values  # Assuming 'output' column is 1 for blue region, 0 for non-blue region

    # Instantiate and run the custom neural network
    model = CustomNN()
    outputs = model.forward(X)

    # Compare the predicted output with the true output
    correct_predictions = np.sum(outputs.flatten() == y_true)
    accuracy = correct_predictions / len(y_true) * 100

    print(f"Accuracy of the neural network on the dataset: {accuracy:.2f}%")

    # Print some sample predictions
    print("\nSample predictions:")
    for i in range(5):  # Show the first 5 predictions
        print(f"Input: {X[i]}, Predicted Output: {outputs[i][0]}, True Output: {y_true[i]}")


Accuracy of the neural network on the dataset: 100.00%

Sample predictions:
Input: [55.59998102 15.70427792], Predicted Output: 1.0, True Output: 1
Input: [38.62130198  1.23693374], Predicted Output: 1.0, True Output: 1
Input: [69.59582739 15.51930083], Predicted Output: 1.0, True Output: 1
Input: [1.60382481 1.48606988], Predicted Output: 0.0, True Output: 0
Input: [84.35581227 98.1395389 ], Predicted Output: 1.0, True Output: 1
